In [17]:
import pandas as pd 
import numpy as np
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [69]:
# Read in dataset
covid_trials_df = pd.read_csv("SearchResults.tsv", sep="\t")

In [70]:
columns_of_interest = ['NCT Number', 
                       'Title', 
                       'Locations',
                       'Status', 
                       'Study Results',
                       'Conditions', 
                       'Interventions', 
                       'Outcome Measures', 
                       'Sponsor/Collaborators', 
                       'Gender', 
                       'Age', 
                       'Phases', 
                       'Enrollment',
                       'Funded Bys', 
                       'Study Type', 
                       'Study Designs',
                       'Start Date',
                       'Completion Date',
                       'First Posted',
                       'Last Update Posted',
                       'URL']
covid_trials_df = covid_trials_df[columns_of_interest]

In [71]:
date_columns = ['Start Date',                       
                'Completion Date',
                'First Posted',
                'Last Update Posted' ]
for d in date_columns:
    # Only keep month and year
    covid_trials_df[d] = [(str(i).split(" ")[0] + " " + str(i).split(" ")[-1]) for i in list(covid_trials_df[d])]

In [72]:
# Add a countries column
covid_trials_df.loc[:,"Location_Country"] = [str(i).split(",")[-1].strip() for i in list(covid_trials_df["Locations"].copy())]
# fix Incorrectly named country columns 
covid_trials_df["Location_Country"].loc[covid_trials_df["Location_Country"] == "Islamic Republic of"] = "Iran"
covid_trials_df["Location_Country"].loc[covid_trials_df["Location_Country"] == "The Democratic Republic of the"] = "Congo"

/opt/conda/lib/python3.7/site-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)


In [73]:
# Add a countries column
covid_trials_df.loc[:,"Location_City_or_State"] = [str(i).split(",")[-2].strip() 
                                                   if len(str(i).split(",")) > 1 
                                                   else str(i)
                                                   for i in list(covid_trials_df["Locations"].copy())]

In [74]:
covid_trials_df.loc[:,"Location_Institution"] = [str(i).split(",")[0].strip()  
                                                   for i in list(covid_trials_df["Locations"].copy())]

# cols_to_explode

In [61]:
# cols_to_explode = ["Interventions", 
#                    "Outcome Measures", 
#                    "Sponsor/Collaborators",
#                    "Funded Bys",
#                    "Study Type",
#                    "Study Designs"]

# cols_not_to_explode = [i for i in list(covid_trials_df.columns) if i not in cols_to_explode]

# # Convert columns with multiple fields into lists so they can be exploded 
# for lst_col in cols_to_explode: 
#     print(lst_col)
#     # split by delimiter
#     covid_trials_df = covid_trials_df.assign(**{lst_col:covid_trials_df[lst_col].str.split('|')})
    
#     # explode delimited column
#     covid_trials_df = covid_trials_df.explode(lst_col)


Interventions
Outcome Measures
Sponsor/Collaborators
Funded Bys
Study Type
Study Designs


In [75]:
covid_trials_df = covid_trials_df.applymap(lambda s:s.upper() if type(s) == str else s)
covid_trials_df = covid_trials_df.replace("nan", np.nan)
covid_trials_df = covid_trials_df.replace("NaN", np.nan)

In [123]:
study_designs = covid_trials_df[['NCT Number', 'Study Designs']].drop_duplicates() # pk: NCT number
interventions = covid_trials_df[['NCT Number', 'Interventions']].drop_duplicates() # pk: NCT number

In [169]:
outcome_measures = covid_trials_df[['NCT Number', 'Outcome Measures']].drop_duplicates() # pk: index
sponsor_collaborators = covid_trials_df[['NCT Number', 'Sponsor/Collaborators']].drop_duplicates() # pk: index
funded_bys = covid_trials_df[['NCT Number', 'Funded Bys']].drop_duplicates() # pk: index
study_type = covid_trials_df[['NCT Number', 'Study Type']].drop_duplicates() # pk: index

In [226]:
trial_into_list = ['NCT Number', 'Title', 'Locations', 'Status', 'Study Results', 'Conditions',
                   'Gender', 'Age', 'Phases','Enrollment', 'URL', 'Location_Country', 'Location_City_or_State',
                   'Location_Institution', 'Start Date', 'Completion Date', 'First Posted','Last Update Posted']
trial_info = covid_trials_df[trial_into_list].drop_duplicates()

# Study Design

In [143]:
study_design_list = ["ALLOCATION", 
                     "INTERVENTION MODEL", 
                     "MASKING",
                     "PRIMARY PURPOSE",
                     "OBSERVATIONAL MODEL",
                     "TIME PERSPECTIVE"] 
def parse_study_design(row):
    dic = {}
    for i in range(len(study_design_list)):
        dic[study_design_list[i]] = 'nan'
    try:
        row = row.split("|")
        for item in row:
            item = item.split(":")
            key = item[0].strip()
            value = item[1].strip()
            if key in dic:
                dic[key] = value
        return dic
    except:
        return dic
study_design_df = study_designs['Study Designs'].apply(parse_study_design)
study_designs.loc[:,study_design_list] = pd.json_normalize(study_design_df)

In [149]:
study_designs.drop('Study Designs', axis=1, inplace=True) 

In [153]:
study_designs = study_designs.replace("nan", np.nan)
study_designs = study_designs.replace("N/A", np.nan)
study_designs.head()

,NCT Number,ALLOCATION,INTERVENTION MODEL,MASKING,PRIMARY PURPOSE,OBSERVATIONAL MODEL,TIME PERSPECTIVE
0,NCT04372602,RANDOMIZED,SINGLE GROUP ASSIGNMENT,"TRIPLE (PARTICIPANT, CARE PROVIDER, INVESTIGATOR)",TREATMENT,NaN,NaN
1,NCT04364698,NaN,NaN,NaN,NaN,COHORT,PROSPECTIVE
2,NCT04482621,RANDOMIZED,PARALLEL ASSIGNMENT,"DOUBLE (PARTICIPANT, INVESTIGATOR)",TREATMENT,NaN,NaN
3,NCT04459637,NaN,NaN,NaN,NaN,COHORT,PROSPECTIVE
4,NCT04425538,NaN,SINGLE GROUP ASSIGNMENT,NONE (OPEN LABEL),TREATMENT,NaN,NaN


# Interventions

In [155]:
intervention_list = ['DRUG', 'PROCEDURE', 'OTHER', 'DEVICE', 'BIOLOGICAL', 'DIAGNOSTIC TEST',
                     'DIETARY SUPPLEMENT', 'GENETIC', 'COMBINATION PRODUCT', 'BEHAVIORAL', 'RADIATION'] 
def parse_intervention(row):
    dic = {}
    for i in range(len(intervention_list)):
        dic[intervention_list[i]] = 'nan'
    try:
        row = row.split("|")
        for item in row:
            item = item.split(":")
            key = item[0].strip()
            value = item[1].strip()
            if key in dic:
                dic[key] = value
        return dic
    except:
        return dic
intervention_df = interventions['Interventions'].apply(parse_intervention)
interventions.loc[:,intervention_list] = pd.json_normalize(intervention_df)

In [157]:
interventions.drop('Interventions', axis=1, inplace=True) 

In [159]:
interventions = interventions.replace("nan", np.nan)
interventions = interventions.replace("N/A", np.nan)
interventions.head()

,NCT Number,DRUG,PROCEDURE,OTHER,DEVICE,BIOLOGICAL,DIAGNOSTIC TEST,DIETARY SUPPLEMENT,GENETIC,COMBINATION PRODUCT,BEHAVIORAL,RADIATION
0,NCT04372602,PLACEBO,PERIPHERAL BLOOD DRAW,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NCT04364698,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NCT04482621,DECITABINE,NaN,PLACEBO SALINE,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NCT04459637,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NCT04425538,INFLIXIMAB,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [77]:
covid_trials_df.head(10)

,NCT Number,Title,Locations,Status,Study Results,Conditions,Interventions,Outcome Measures,Sponsor/Collaborators,Gender,Age,Phases,Enrollment,Funded Bys,Study Type,Study Designs,Start Date,Completion Date,First Posted,Last Update Posted,URL,Location_Country,Location_City_or_State,Location_Institution
0,NCT04372602,DUVELISIB TO COMBAT COVID-19,"WASHINGTON UNIVERSITY SCHOOL OF MEDICINE, SAIN...",NOT YET RECRUITING,NO RESULTS AVAILABLE,COVID-19,DRUG: DUVELISIB|PROCEDURE: PERIPHERAL BLOOD DR...,OVERALL SURVIVAL|LENGTH OF HOSPITAL STAY|LENGT...,WASHINGTON UNIVERSITY SCHOOL OF MEDICINE|VERAS...,ALL,"18 YEARS AND OLDER (ADULT, OLDER ADULT)",PHASE 2,28.0,OTHER|INDUSTRY,INTERVENTIONAL,ALLOCATION: RANDOMIZED|INTERVENTION MODEL: SIN...,SEPTEMBER 2020,MARCH 2022,MAY 2020,SEPTEMBER 2020,HTTPS://CLINICALTRIALS.GOV/SHOW/NCT04372602,UNITED STATES,MISSOURI,WASHINGTON UNIVERSITY SCHOOL OF MEDICINE
1,NCT04364698,OBSERVATIONAL COHORT OF COVID-19 PATIENTS AT R...,"DEPARTMENT OF INFECTIOLOGY, RAYMOND POINCARÉ H...",RECRUITING,NO RESULTS AVAILABLE,COVID-19,NaN,"CLINICAL, BIOLOGICAL AND RADIOLOGICAL CHARACTE...",ASSISTANCE PUBLIQUE - HÔPITAUX DE PARIS,ALL,"18 YEARS AND OLDER (ADULT, OLDER ADULT)",NaN,500.0,OTHER,OBSERVATIONAL,OBSERVATIONAL MODEL: COHORT|TIME PERSPECTIVE: ...,MAY 2020,JUNE 2020,APRIL 2020,MAY 2020,HTTPS://CLINICALTRIALS.GOV/SHOW/NCT04364698,FRANCE,GARCHES,DEPARTMENT OF INFECTIOLOGY
2,NCT04482621,DECITABINE FOR CORONAVIRUS (COVID-19) PNEUMONI...,"JOHNS HOPKINS UNIVERSITY, BALTIMORE, MARYLAND,...",RECRUITING,NO RESULTS AVAILABLE,COVID-19,DRUG: DECITABINE|OTHER: PLACEBO SALINE,CHANGE IN CLINICAL STATE AS ASSESSED BY A 6-PO...,JOHNS HOPKINS UNIVERSITY,ALL,"18 YEARS AND OLDER (ADULT, OLDER ADULT)",PHASE 2,40.0,OTHER,INTERVENTIONAL,ALLOCATION: RANDOMIZED|INTERVENTION MODEL: PAR...,AUGUST 2020,JULY 2021,JULY 2020,AUGUST 2020,HTTPS://CLINICALTRIALS.GOV/SHOW/NCT04482621,UNITED STATES,MARYLAND,JOHNS HOPKINS UNIVERSITY
3,NCT04459637,COVID-19 SURVEILLANCE BASED ON SMART WEARABLE ...,"PEKING UNIVERSITY FIRST HOSPITAL, BEIJING, BEI...",NOT YET RECRUITING,NO RESULTS AVAILABLE,COVID-19,NaN,DETERIORATION OF THE CONDITION|MORTALITY|THE I...,PEKING UNIVERSITY FIRST HOSPITAL,ALL,"18 YEARS TO 75 YEARS (ADULT, OLDER ADULT)",NaN,200.0,OTHER,OBSERVATIONAL,OBSERVATIONAL MODEL: COHORT|TIME PERSPECTIVE: ...,JULY 2020,MARCH 2021,JULY 2020,JULY 2020,HTTPS://CLINICALTRIALS.GOV/SHOW/NCT04459637,CHINA,BEIJING,PEKING UNIVERSITY FIRST HOSPITAL
4,NCT04425538,A PHASE 2 TRIAL OF INFLIXIMAB IN CORONAVIRUS D...,"TUFTS MEDICAL CENTER, BOSTON, MASSACHUSETTS, U...",RECRUITING,NO RESULTS AVAILABLE,COVID-19,DRUG: INFLIXIMAB,TIME TO IMPROVEMENT IN OXYGENATION|28-DAY MORT...,TUFTS MEDICAL CENTER|NATIONAL INSTITUTES OF HE...,ALL,"18 YEARS AND OLDER (ADULT, OLDER ADULT)",PHASE 2,17.0,OTHER|NIH,INTERVENTIONAL,ALLOCATION: N/A|INTERVENTION MODEL: SINGLE GRO...,JUNE 2020,DECEMBER 2020,JUNE 2020,JUNE 2020,HTTPS://CLINICALTRIALS.GOV/SHOW/NCT04425538,UNITED STATES,MASSACHUSETTS,TUFTS MEDICAL CENTER
5,NCT04547257,SAFETY & PERFORMANCE EVALUATION OF SERAPH 100 ...,"JENS SCHIRRMACHER, VAALS, LIMBURG, NETHERLANDS",NOT YET RECRUITING,NO RESULTS AVAILABLE,COVID-19,DEVICE: SERAPH 100,CHANGE IN ORGAN FAILURE|CHANGE IN ORGAN FAILUR...,EXTHERA MEDICAL EUROPE BV|EXTHERA MEDICAL CORP...,ALL,"18 YEARS TO 90 YEARS (ADULT, OLDER ADULT)",NOT APPLICABLE,42.0,INDUSTRY,INTERVENTIONAL,ALLOCATION: RANDOMIZED|INTERVENTION MODEL: PAR...,OCTOBER 2020,APRIL 2021,SEPTEMBER 2020,SEPTEMBER 2020,HTTPS://CLINICALTRIALS.GOV/SHOW/NCT04547257,NETHERLANDS,LIMBURG,JENS SCHIRRMACHER
6,NCT04454307,SAFETY AND EFFICACY OF TRAMADOL IN COVID-19 EG...,NaN,NOT YET RECRUITING,NO RESULTS AVAILABLE,COVID-19,DRUG: TRAMADOL|OTHER: STANDARD CARE DELIVERED ...,NUMBER OF COVID-19 PCR NEGATIVE CASES,TANTA UNIVERSITY,ALL,"18 YEARS TO 65 YEARS (ADULT, OLDER ADULT)",PHASE 1|PHASE 2,100.0,OTHER,INTERVENTIONAL,ALLOCATION: RANDOMIZED|INTERVENTION MODEL: PAR...,JULY 2020,OCTOBER 2020,JULY 2020,JULY 2020,HTTPS://CLINICALTRIALS.GOV/SHOW/NCT04454307,

# Creating a SQLite3 schema following 3NF

In [80]:
def checkPK(df, pk):
    if np.any(df[pk].isnull()):
        print('NULL values.')
    elif df[pk].drop_duplicates().shape[0] != df.shape[0]:
        print('Duplication.')
    else:
        print('Valid PK.')

In [81]:
checkPK(covid_trials_df, 'NCT Number')

Valid PK.


In [82]:
import sqlite3
conn = sqlite3.connect('covid_trials.sqlite')

## tables: study_designs, interventions, outcome_measures, sponsor_collaborators, funded_bys, study_type, trial_info

In [218]:
outcome_measures = covid_trials_df[['NCT Number', 'Outcome Measures']].drop_duplicates() # pk: index
sponsor_collaborators = covid_trials_df[['NCT Number', 'Sponsor/Collaborators']].drop_duplicates() # pk: index
funded_bys = covid_trials_df[['NCT Number', 'Funded Bys']].drop_duplicates() # pk: index
study_type = covid_trials_df[['NCT Number', 'Study Type']].drop_duplicates() # pk: index

In [219]:
outcome_measures = outcome_measures.assign(**{'Outcome Measures':outcome_measures['Outcome Measures'].str.split('|')})
outcome_measures = outcome_measures.explode('Outcome Measures')
sponsor_collaborators = sponsor_collaborators.assign(**{'Sponsor/Collaborators':sponsor_collaborators['Sponsor/Collaborators'].str.split('|')})
sponsor_collaborators = sponsor_collaborators.explode('Sponsor/Collaborators')
funded_bys = funded_bys.assign(**{'Funded Bys':funded_bys['Funded Bys'].str.split('|')})
funded_bys = funded_bys.explode('Funded Bys')
study_type = study_type.assign(**{'Study Type':study_type['Study Type'].str.split('|')})
study_type = study_type.explode('Study Type')

In [214]:
outcome_measures.reset_index(drop=True, inplace=True)
outcome_measures['index'] = outcome_measures.index
outcome_measures.head()

,NCT Number,Outcome Measures,index
0,NCT04372602,OVERALL SURVIVAL,0
1,NCT04372602,LENGTH OF HOSPITAL STAY,1
2,NCT04372602,LENGTH OF ICU STAY,2
3,NCT04372602,DURATION OF VENTILATOR USE,3
4,NCT04372602,DURATION OF VASOPRESSORS USE,4


In [216]:
sponsor_collaborators.reset_index(drop=True, inplace=True)
sponsor_collaborators['index'] = sponsor_collaborators.index
sponsor_collaborators.head()

,NCT Number,Sponsor/Collaborators,index
0,NCT04372602,WASHINGTON UNIVERSITY SCHOOL OF MEDICINE,0
1,NCT04372602,"VERASTEM, INC.",1
2,NCT04364698,ASSISTANCE PUBLIQUE - HÔPITAUX DE PARIS,2
3,NCT04482621,JOHNS HOPKINS UNIVERSITY,3
4,NCT04459637,PEKING UNIVERSITY FIRST HOSPITAL,4


In [220]:
funded_bys.reset_index(drop=True, inplace=True)
funded_bys['index'] = funded_bys.index
funded_bys.head()

,NCT Number,Funded Bys,index
0,NCT04372602,OTHER,0
1,NCT04372602,INDUSTRY,1
2,NCT04364698,OTHER,2
3,NCT04482621,OTHER,3
4,NCT04459637,OTHER,4


In [221]:
study_type.reset_index(drop=True, inplace=True)
study_type['index'] = study_type.index
study_type.head()

,NCT Number,Study Type,index
0,NCT04372602,INTERVENTIONAL,0
1,NCT04364698,OBSERVATIONAL,1
2,NCT04482621,INTERVENTIONAL,2
3,NCT04459637,OBSERVATIONAL,3
4,NCT04425538,INTERVENTIONAL,4


In [223]:
study_designs.to_sql('study_designs', conn, if_exists='replace', index=False)
interventions.to_sql('interventions', conn, if_exists='replace', index=False)
trial_info.to_sql('trial_info', conn, if_exists='replace', index=False)
outcome_measures.to_sql('outcome_measures', conn, if_exists='replace', index=False)
sponsor_collaborators.to_sql('sponsor_collaborators', conn, if_exists='replace', index=False)
funded_bys.to_sql('funded_bys', conn, if_exists='replace', index=False)
study_type.to_sql('study_type', conn, if_exists='replace', index=False)